In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('Bitcoin_tweets.csv', low_memory=False)
data

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4236176,需要粉丝的进主页查询,NaN,"""自助下单 网址： https://t.co/hlxL9g2Ey6\n推特，ins，yout...",2017-11-18 17:52:13,12,10,7,False,2022-09-14 11:40:27,and you #粉丝 #twitter粉丝 #twitter刷粉 #blockchai...,"['粉丝', 'twitter粉丝', 'twitter刷粉', 'blockchain',...",Twitter Web App,False
4236177,Dr. Robin Kiera,Hamburg,"#Digitalization #insurtech, #fintech #blockcha...",2011-01-27 10:03:57,42788,12778,27052,True,2022-09-14 11:40:23,#Cryptocurrency #fraud sparks concerns over ad...,"['Cryptocurrency', 'fraud', 'tech', 'DigitalAs...",SocialPilot.co,False
4236178,All Things Defi,NaN,"#Defi lover, crypto hodler.",2022-08-02 05:23:00,115,5,0,False,2022-09-14 11:40:18,"updated price for $LTC, $AVAX, $SOL:\n\n#LTC $...","['LTC', 'AVAX', 'BTC', 'SOL', 'ETH', 'cryptocu...",Twitter Web App,False
4236179,Meta guy,United States,#Crypto and #Metaverse,2021-06-05 08:00:00,70,22,121,False,2022-09-14 11:40:15,When #btc goes down LUNA CLASSIC goes up..\n\n...,"['btc', 'LUNC', 'LUNCARMY']",Twitter for Android,False


In [3]:
data = data[['date', 'text']]
data

,date,text
0,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...
1,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""..."
2,2021-02-10 23:54:48,"Guys evening, I have read this article about B..."
3,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...
4,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...
...,...,...
4236176,2022-09-14 11:40:27,and you #粉丝 #twitter粉丝 #twitter刷粉 #blockchai...
4236177,2022-09-14 11:40:23,#Cryptocurrency #fraud sparks concerns over ad...
4236178,2022-09-14 11:40:18,"updated price for $LTC, $AVAX, $SOL:\n\n#LTC $..."
4236179,2022-09-14 11:40:15,When #btc goes down LUNA CLASSIC goes up..\n\n...


In [4]:
data.isna().sum()

date    8
text    8
dtype: int64

In [5]:
data = data.dropna()
data.isna().sum()

date    0
text    0
dtype: int64

In [6]:
from cmath import nan
from datetime import datetime

def str_to_date(d):
    try:
        return datetime.strptime(d.split(' ')[0], '%Y-%m-%d')
    except:
        return nan

In [7]:
x = np.array([str_to_date(d) for d in data.date])
data = data.drop(columns='date')
data.insert(0, 'date', x)
data

,date,text
0,2021-02-10,Blue Ridge Bank shares halted by NYSE after #b...
1,2021-02-10,"😎 Today, that's this #Thursday, we will do a ""..."
2,2021-02-10,"Guys evening, I have read this article about B..."
3,2021-02-10,$BTC A big chance in a billion! Price: \487264...
4,2021-02-10,This network is secured by 9 508 nodes as of t...
...,...,...
4236176,2022-09-14,and you #粉丝 #twitter粉丝 #twitter刷粉 #blockchai...
4236177,2022-09-14,#Cryptocurrency #fraud sparks concerns over ad...
4236178,2022-09-14,"updated price for $LTC, $AVAX, $SOL:\n\n#LTC $..."
4236179,2022-09-14,When #btc goes down LUNA CLASSIC goes up..\n\n...


In [8]:
data.isna().sum()

date    17
text     0
dtype: int64

In [9]:
data = data.dropna()
data = data.sort_values(by='date')
data.reset_index(drop=True, inplace=True)
data

,date,text
0,2021-02-05,🎲 Lucky Spin Giveaway\n\n💰 Reward up to 1 #BTC...
1,2021-02-05,"$BTC #BTC #Bitcoin: $37,508.17\n #tradealert \..."
2,2021-02-05,#Bitcoin #BTC $MARA $RIOT #inflation $GBTC #Cr...
3,2021-02-05,“you look like all these btc boys”. \nWhat doe...
4,2021-02-05,Don't be fooled guys #socialmedia is next.. th...
...,...,...
4236151,2022-09-14,"With #Bitcoin dumping and $LUNC fud, at what p..."
4236152,2022-09-14,Current #Cardano price Alert: \n\n#ADA $0.47 U...
4236153,2022-09-14,^^Bitcoin is not:\nA crypto republic\nCrypto d...
4236154,2022-09-14,Successful #generativeAI examples worth noting...


In [10]:
def check_skipped_days():
    x = 0
    arr = np.array([])
    day_before = datetime.now()
    index_before = 0

    for index, d in enumerate(data.date):
        diff = 0

        if x == 0:
            day_before, index_before = d, index
            x+=1
            continue

        diff = (d - day_before).days

        if diff > 1:
            arr = np.append(arr, '{} => {} : {} days'.format(index_before, index, diff))

        day_before, index_before = d, index

    return arr

In [11]:
skipped_days = check_skipped_days()

print(skipped_days)
len(skipped_days)

['21524 => 21525 : 3 days' '32469 => 32470 : 3 days'
 '39244 => 39245 : 3 days' '42808 => 42809 : 6 days'
 '44442 => 44443 : 11 days' '48582 => 48583 : 24 days'
 '73739 => 73740 : 5 days' '106642 => 106643 : 31 days'
 '128424 => 128425 : 22 days' '254219 => 254220 : 9 days'
 '346506 => 346507 : 11 days' '720298 => 720299 : 5 days'
 '851404 => 851405 : 5 days' '1085668 => 1085669 : 2 days'
 '1209285 => 1209286 : 15 days' '1232795 => 1232796 : 38 days'
 '1489456 => 1489457 : 4 days' '1584591 => 1584592 : 6 days'
 '1691909 => 1691910 : 5 days' '1751616 => 1751617 : 6 days'
 '1842615 => 1842616 : 5 days' '1944221 => 1944222 : 15 days'
 '1950225 => 1950226 : 6 days' '1970664 => 1970665 : 12 days'
 '1999522 => 1999523 : 12 days' '2126399 => 2126400 : 5 days'
 '2259609 => 2259610 : 18 days' '2284912 => 2284913 : 5 days'
 '2339084 => 2339085 : 12 days' '2347458 => 2347459 : 4 days'
 '2433889 => 2433890 : 4 days' '2699142 => 2699143 : 20 days'
 '3028940 => 3028941 : 5 days' '3199720 => 3199721 

43

In [12]:
data = data.loc[4132303:]
data.reset_index(drop=True, inplace=True)

In [13]:
skipped_days = check_skipped_days()

print(skipped_days)
len(skipped_days)

[]


0

In [14]:
print(data.date.min())
print(data.date.max())
print((data.date.max() - data.date.min()).days)

2022-09-11 00:00:00
2022-09-14 00:00:00
3
